In [2]:
import ROOT

# This script has three purposes
# - Debugging G4 hits for HGTD and connecting to the truth record
# - Making simple low-stats plots to understand performance
# - Dumping out event display info for Phoenix at https://hepsoftwarefoundation.org/phoenix/#/atlas
# The script is *not* meant for large-scale processing as it's too slow for that

# imports
import json
import math
import colorsys
# if installed, use the particles module to print out particle names
# https://pypi.org/project/particle/
doParticleNames = True
try:
    import particle
except ImportError:
    print('\nThe package "particle" was not found, will not print particle names (see https://pypi.org/project/particle/ to install)')
    doParticleNames = False

# options
writeEventDisplayJSON = True
printDetailsOfTruthMatch = True
printDecayTracingLevel = 5
printDetailsEventMax = 3

doPlots = True
outFileName = "HGTD_plots.root"

# set up xAOD interface
if(not ROOT.xAOD.Init().isSuccess()): print("Failed xAOD.Init()")

# load a file and retrieve the tree
f = ROOT.TFile.Open("DAOD_TRUTH0.HGTD_ttbar_test.pool.root", "READ")
tree = ROOT.xAOD.MakeTransientTree(f, "CollectionTree")

# the CollectionTree is an xAOD::TEventTree, which inherits from TTree,
# so we can add a friend to it - e.g. an ntuple dumped out by the SiAnalysis alg!
tree.AddFriend("SiHGTD", "SiHitValid_s3654.root")
hgtdHits = tree.GetFriend("SiHGTD")

# uncomment to see the list of branches we have in the main and friended trees
#for b in tree.GetListOfBranches():
#    print(b.GetName())
#for b in hgtdHits.GetListOfBranches():
#    print(b.GetName())

# event display dict to be dumped to JSON
edOutput = {}
edCounter = 0

# pion mass
mPi = 139.57 # in MeV
c = 2.99792458e11 # in mm/s

# get pretty names for particles if the modules needed exist
def getPdgIdString(tp):
    pdgIdString = "%d" % tp.pdgId()
    if doParticleNames:
        try:
            pdgIdString += " (%s)" % particle.Particle.from_pdgid(tp.pdgId()).name
        except particle.particle.particle.ParticleNotFound:
            pdgIdString += " (unknown)"
    return pdgIdString

# print truth particle info
def tpInfo(tp, vtxInfo = False, prodDetails = False, decayDetails = False, nSpaces = 0):

    string = "PDG id: %s, barcode: %d, status: %d, (pT,eta,phi,m) = (%.2f,%.2f,%.2f,%.3g)" % (getPdgIdString(tp), tp.barcode(), tp.status(), tp.pt()/1000, tp.eta(), tp.phi(), tp.m()/1000.)
    if vtxInfo:
        if tp.hasProdVtx():
            string += (", prod at (%.2f,%.2f,%.2f mm, %.3g ns)" % (tp.prodVtx().x(), tp.prodVtx().y(), tp.prodVtx().z(), tp.prodVtx().t()/(c*1e-9)))
            if prodDetails:
                string += (" nIn: %d, nOut: %d" % (tp.prodVtx().nIncomingParticles(), tp.prodVtx().nOutgoingParticles()))
        if tp.hasDecayVtx():
            string += (", decay at (%.2f,%.2f,%.2f mm, %3g ns)" % (tp.decayVtx().x(), tp.decayVtx().y(), tp.decayVtx().z(), tp.decayVtx().t()/(c*1e-9)))
            if decayDetails:
                string += (" nIn: %d, nOut: %d" % (tp.decayVtx().nIncomingParticles(), tp.decayVtx().nOutgoingParticles()))
        if tp.hasProdVtx() and tp.hasDecayVtx():
            p = tp.prodVtx()
            d = tp.decayVtx()
            tdiff = (d.t() - p.t())/(c*1e-9)
            string += (" -> traveled %.3g mm (in %.3g ns)" % ((d.v4().Vect()-p.v4().Vect()).Mag(), tdiff))
    return (nSpaces*" ")+string

# create string with info about a hit in HGTD
def hitInfo(hitIndex):
    string = "HGTD hit: (x,y,z) = (%.1f,%.1f,%.1f) mm, t = %.3f ns" % (
        hgtdHits.HGTD_x[h],
        hgtdHits.HGTD_y[h],
        hgtdHits.HGTD_z[h],
        hgtdHits.HGTD_time[h]
        )
    return string

# create some plots to be filled when looping over events
plots = {}
categories = ["G4", "Gen"]

outFile = ROOT.TFile(outFileName, "RECREATE")
if doPlots:
    outFile.cd()
    for cat in categories:
        plots["Number of HGTD hits %s" % cat] = ROOT.TH1F("Number of HGTD hits %s" % cat, "Number of HGTD hits %s" % cat, 6, 0, 6)
        plots["Number of HGTD hits 2 GeV %s" % cat] = ROOT.TH1F("Number of HGTD hits 2 GeV %s" % cat, "Number of HGTD hits 2 GeV %s" % cat, 6, 0, 6)
        plots["Number of HGTD hits vs. pT %s" % cat] = ROOT.TProfile("Number of HGTD hits vs. pT %s" % cat, "Number of HGTD hits vs. pT %s" % cat, 50, 0, 10)
        plots["Number of HGTD hits vs. pT 2D %s" % cat] = ROOT.TH2F("Number of HGTD hits vs. pT 2D %s" % cat, "Number of HGTD hits vs. pT 2D %s" % cat, 20, 0, 10, 5, 0, 5)
        plots["Number of HGTD hits vs. eta %s" % cat] = ROOT.TProfile("Number of HGTD hits vs. eta %s" % cat, "Number of HGTD hits vs. eta %s" % cat, 50, 2.2, 4.2)
        plots["Number of HGTD hits vs. eta 2D %s" % cat] = ROOT.TH2F("Number of HGTD hits vs. eta 2D %s" % cat, "Number of HGTD hits vs. eta 2D %s" % cat, 20, 2.2, 4.2, 5, 0, 5)

    # TODO: add more plots for different types of particles?

# to color-code the tracks in the output JSON file for Phoenix
chargeColors = {
    -1: "0x0000ff",
    0: "0x00ff00",
    1: "0xff0000"
}

# loop over the events!
eventCount = 0
maxEvents = 9 # t.GetEntries()
for e in tree:

    # print out the first few, and every nth
    printEventDetails = (eventCount < printDetailsEventMax) or (eventCount % 10 == 0)

    if eventCount >= maxEvents:
        print("eventCount = %d --> stopping!" % eventCount)
        break

    print("Event number: %d (eventCount = %d, printEventDetails = %s)" % (e.EventInfo.eventNumber(), eventCount, str(printEventDetails)))
    if printEventDetails:
        print("  Number of truth particles: %d" % len(e.TruthParticles))
        print("  Number of HGTD hits: %d" % len(hgtdHits.HGTD_x))

    # keep track of HGTD hits for each truth particle, a list of hit indices is stored for each truth particle index (the TruthParticle doesn't work as key, unfortunately)
    particlesWithHgtdHits = {}

    # find the truth particles that have HGTD hits
    for tpIndex,tp in enumerate(e.TruthParticles):

        # debug printout
        #if printEventDetails:
            #print("Truth: " + tpInfo(tp, True, True, True))

        # skip non-stable, neutrals and those outside of acceptance
        if tp.status() != 1 or tp.charge == 0 or abs(tp.eta()) < 2.3 or abs(tp.eta()) > 4.1:
            continue
        # TODO: skip those with too low pT to reach the HGTD
        # and those that decay before the HGTD
        if tp.hasDecayVtx():
            if abs(tp.decayVtx().z()) < 3430:
                continue

        # for the rest, see if the barcode of any of the HGTD hits matches the barcode of the tp
        bc = tp.barcode()
        #print("Will look for HGTD hit for this truth particle:")
        if printEventDetails:
            print(tpInfo(tp, True, True, True))
        matchedHitIndices = []
        for h in range(0, len(hgtdHits.HGTD_barcode)):
            if bc == hgtdHits.HGTD_barcode[h]:
                if printEventDetails:
                    print("  Match! " + hitInfo(h))
                matchedHitIndices.append(h)
        if len(matchedHitIndices) > 0:
            particlesWithHgtdHits[tpIndex] = matchedHitIndices

        # a truth particle is either created by the generator or G4
        catStr = "Gen"
        if bc > 2e5:
            catStr = "G4"

        # now fill the plots
        if doPlots:
            # fill highest pT bin also for tracks with higher pT
            fillpT = min(9.9, tp.pt()/1000)
            plots["Number of HGTD hits %s" % catStr].Fill(len(matchedHitIndices))
            if tp.pt() > 2e3:
                plots["Number of HGTD hits 2 GeV %s" % catStr].Fill(len(matchedHitIndices))
            plots["Number of HGTD hits vs. pT %s" % catStr].Fill(fillpT, len(matchedHitIndices))
            plots["Number of HGTD hits vs. pT 2D %s" % catStr].Fill(fillpT, len(matchedHitIndices))
            plots["Number of HGTD hits vs. eta %s" % catStr].Fill(abs(tp.eta()), len(matchedHitIndices))
            plots["Number of HGTD hits vs. eta 2D %s" % catStr].Fill(abs(tp.eta()), len(matchedHitIndices))

    # fill in some ED data
    if writeEventDisplayJSON:
        edOutput["Event %d" % eventCount] = {}
        edOutput["Event %d" % eventCount]["event number"] = e.EventInfo.eventNumber()
        edOutput["Event %d" % eventCount]["run number"] = e.EventInfo.mcChannelNumber()

    # add the decay vertices of the found LLP decays, and the relevant truth particles (LLP + decay products)
    if writeEventDisplayJSON:
        edOutput["Event %d" % eventCount]["Hits"] = {}
        edOutput["Event %d" % eventCount]["Hits"]["HGTD hits"] = []
        edOutput["Event %d" % eventCount]["Tracks"] = {}
        edOutput["Event %d" % eventCount]["Tracks"]["Truth particles"] = []
        #print("edOutput:")
        #print(edOutput)
        tpList = []
        for tpIndex,tp in enumerate(e.TruthParticles):
            # skip truth particles that are not stable or do not have a production vertex
            if (tp.status() != 1) or (not tp.hasProdVtx()):
                continue
            # also skip truth particles produced at |z| > 4m
            if abs(tp.prodVtx().z()) > 4000:
                continue
            #print(tpInfo(tp, True, True, True))
            # if there is no decay vertex for the truth particle, we need to extrapolate it
            tmpVec = ROOT.TLorentzVector()
            tmpVec.SetPtEtaPhiM( 0.4*math.log10(tp.pt()), tp.eta(), tp.phi(), tp.m() )
            tpDict = {
                "color": chargeColors[int(tp.charge())],
                "pdgId": getPdgIdString(tp),
                "barcode": tp.barcode(),
                "prodVtx": "(%.2f, %.2f, %.2f)" % (tp.prodVtx().x(), tp.prodVtx().y(), tp.prodVtx().z()),
                "(pT, eta, phi, m)" : "(%.2f, %.2f, %.2f, %.2f)" % (tp.pt(), tp.eta(), tp.phi(), tp.m()),
                "pos": [
                    [ tp.prodVtx().x(), tp.prodVtx().y(), tp.prodVtx().z() ],
                    [ tp.prodVtx().x(), tp.prodVtx().y(), tp.prodVtx().z() ]
                ]
                }
            # if there are matched HGTD hits, use those as additional points on the track
            if tpIndex in particlesWithHgtdHits:
                nHits = len(particlesWithHgtdHits[tpIndex])
                #if printEventDetails:
                #    print("Last hit(s) from %d HGTD hits!" % nHits)
                tpDict["HGTD hits"] = nHits
                for h in range(0, nHits):
                    hitIndex = particlesWithHgtdHits[tpIndex][h]
                    tpDict["pos"].append( [ hgtdHits.HGTD_x[hitIndex], hgtdHits.HGTD_y[hitIndex], hgtdHits.HGTD_z[hitIndex] ] )
                #if printEventDetails:
                #    print("Added HGTD hits as points to track:")
                #    print(tpDict)
            # if the particle has a decay vertex, use it as end point
            elif tp.hasDecayVtx():
                #print("Last hit from decay vertex!")
                tpDict["decayVtx"] = ("(%.2f, %.2f, %.2f)" % (tp.decayVtx().x(), tp.decayVtx().y(), tp.decayVtx().z()))
                tpDict["pos"].append( [ tp.decayVtx().x(), tp.decayVtx().y(), tp.decayVtx().z() ] )
            # if neither exist, extend based on momentum vector
            else:
                #print("Last hit from extrapolation!")
                tpDict["pos"].append( [ tp.prodVtx().x()+tmpVec.X(), tp.prodVtx().y()+tmpVec.Y(), tp.prodVtx().z()+tmpVec.Z() ] )
            edOutput["Event %d" % eventCount]["Tracks"]["Truth particles"].append(tpDict)

        for h in range(0, len(hgtdHits.HGTD_x)):
            # add LLP decay vertices
            edOutput["Event %d" % eventCount]["Hits"]["HGTD hits"].append(
                [hgtdHits.HGTD_x[h], hgtdHits.HGTD_y[h], hgtdHits.HGTD_z[h]]
            )

    eventCount += 1

if writeEventDisplayJSON:
    with open('HGTD_events.json', 'w') as outfile:
        json.dump(edOutput, outfile, indent=4)

# make sure to write out the plots
if doPlots:
    outFile.cd()
    for p in plots:
        plots[p].Write()
    outFile.Close()

ROOT.xAOD.ClearTransientTrees()


The package "particle" was not found, will not print particle names (see https://pypi.org/project/particle/ to install)


AttributeError: Failed to get attribute xAOD from ROOT